<a href="https://colab.research.google.com/github/techwithanirudh/Shared/blob/master/DeepFake_Website_Part4_AlmostDone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.mkdir('/content/DeepFake')
os.mkdir('/content/DeepFake/templates')
os.chdir('/content/DeepFake')

In [2]:
!pip install flask_ngrok flask_cors

In [3]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
!cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 72.13 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [4]:
from google.colab import drive
drive.mount('/content/DeepFake/gdrive')

Mounted at /content/DeepFake/gdrive


In [5]:
# os.chdir('/content/gdrive/MyDrive')
# !unzip first-order-motion-model.zip
os.chdir('/content/DeepFake/first-order-model')

**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.
Alternativelly you can use this mirror link https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing**

Or download the zip to your computer and put it to your drive, then mount the drive

In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML, Image
from IPython.display import display as display_ipy
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [8]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                                          checkpoint_path='/content/DeepFake/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

In [9]:
with open('/content/DeepFake/templates/index.html', 'w') as f:
  f.write('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <title>Upload Files</title>
    <style>
        body{
          background: rgba(0,0,0,0.9);
        }
        form{
          position: absolute;
          top: 50%;
          left: 50%;
          margin-top: -100px;
          margin-left: -250px;
          width: 500px;
          height: 200px;
          border: 4px dashed #fff;
        }
        form p{
          width: 100%;
          height: 100%;
          text-align: center;
          line-height: 170px;
          color: #ffffff;
          font-family: Arial;
        }
        form input{
          position: absolute;
          margin: 0;
          padding: 0;
          width: 100%;
          height: 100%;
          outline: none;
          opacity: 0;
        }
        form button{
          margin: 0;
          color: #fff;
          background: #16a085;
          border: none;
          width: 508px;
          height: 35px;
          margin-top: -20px;
          margin-left: -4px;
          border-radius: 4px;
          border-bottom: 4px solid #117A60;
          transition: all .2s ease;
          outline: none;
        }
        form button:hover{
          background: #149174;
        	color: #0C5645;
        }
        form button:active{
          border:0;
        }
    </style>
</head>
<body>
    <form action="/upload" method="POST" enctype="multipart/form-data">
        <input type="file" name="file" multiple>
        <p>Drag your files here or click in this area.</p>
        <button type="submit">Upload</button>
    </form>

    <script>
        $(document).ready(function(){
          $('form input').change(function () {
            $('form p').text(this.files.length + " file(s) selected");
          });
        });
    </script>
</body>
</html>
  ''')
f.close()

In [10]:
with open('/content/DeepFake/templates/progress.html', 'w') as f:
  f.write('''

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Progress</title>
    <style>
        body{
          background: rgba(0,0,0,0.9);
        }
        .progress-line, .progress-line:before {
          height: 20px;
          width: 100%;
          margin: 0;    
        }
        .progress-line {
          background-color: #b3d4fc;
          display: -webkit-flex;
          display: flex;
        }
        .progress-line:before {
          background-color: #3f51b5;
          content: '';
          -webkit-animation: running-progress 2s cubic-bezier(0.4, 0, 0.2, 1) infinite;
          animation: running-progress 2s cubic-bezier(0.4, 0, 0.2, 1) infinite;
        }
        @-webkit-keyframes running-progress {
          0% { margin-left: 0px; margin-right: 100%; }
          50% { margin-left: 25%; margin-right: 0%; }
          100% { margin-left: 100%; margin-right: 0; }
        }
        @keyframes running-progress {
          0% { margin-left: 0px; margin-right: 100%; }
          50% { margin-left: 25%; margin-right: 0%; }
          100% { margin-left: 100%; margin-right: 0; }
        }
    </style>
</head>
<body>
    <div class="main">
      <h2 style="color: white;">Creating DeepFake</h2>
      <div class="progress-line"></div>
    <div>
    <script>
      document.location.href = '/convertDf';
    </script>
</body>
</html>
  ''')
f.close()

In [11]:
with open('/content/DeepFake/templates/jsCheck.html', 'w') as f:
  f.write('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>JsCheck</title>
</head>
<body>
    <noscript>
        <div class="noscriptmsg">
            <h1>
                JavaScript must be enabled in order for you to use This website. However, it seems JavaScript is either disabled or not supported by your browser. 
                To use continue to this website, enable JavaScript by changing your browser options, then try again.
            </h1>
        </div>
    </noscript>
    <script>
        document.location.href = '/main'
    </script>
</body>
</html>
  ''')
f.close()

In [16]:
reader = imageio.get_reader('/content/DeepFake/gdrive/My Drive/first-order-motion-model/04.mp4')
fps = reader.get_meta_data()['fps']

In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from werkzeug.utils import secure_filename
import time, requests

app = Flask(__name__, root_path='/content/DeepFake')
CORS(app)
run_with_ngrok(app)

@app.route('/')
def ip_get():
  global jsonIp
  if request.environ.get('HTTP_X_FORWARDED_FOR') is None:
    jsonIp = {'ip': request.environ['REMOTE_ADDR']}
  else:
    jsonIp = {'ip': request.environ['HTTP_X_FORWARDED_FOR']}

  print(jsonIp)
  return redirect('/jsCheck')

@app.route('/jsCheck')
def jsCheck():
  return render_template('jsCheck.html')

@app.route('/main')
def main():
  return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
  f = request.files['file']
  f.save('/content/DeepFake/face.png')
  return render_template('progress.html')

@app.route('/convertDf', methods=['GET', 'POST'])
def deepFake():
  source_image = imageio.imread('/content/DeepFake/face.png')
  driving_video = imageio.mimread('/content/DeepFake/gdrive/My Drive/first-order-motion-model/10.mp4', memtest=False)

  #Resize image and video to 256x256

  source_image = resize(source_image, (256, 256))[..., :3]
  driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

  predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                               adapt_movement_scale=True)

  imageio.mimsave('/content/DeepFake/generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
  return send_file('/content/DeepFake/generated.mp4', mimetype='video/mp4')

app.run()

In [5]:
!hostname -I
import shutil

shutil.rmtree('/content/DeepFake')

172.28.0.2 
